This is for informational purposes and wont be able to run, as that requires the data to be sent as well

However it is a general script that shows how general our code is as we raelly just need ot change the model and datalaoder to be able to run a different model here

In [ ]:
import sys
sys.path.insert(0,"/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/") 
__package__ = "GeneralNotebook_ForHow_CodeWorks"

In [ ]:
from SingleModels.train_model.text_training import train_text_network, evaluate_text
from SingleModels.models.text import BertClassifier
import wandb

from utils.data_loaders import BertDataset
from utils.global_functions import arg_parse 
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader
from utils.global_functions import arg_parse , hidden_layer_count , Metrics
from sklearn.model_selection import train_test_split

In [ ]:
def prepare_dataloader(df , batch_size, pin_memory=True, num_workers=4):
    """
    we load in our dataset, and we just make a random distributed sampler to evenly partition our 
    dataset on each GPU
    say we have 32 data points, if batch size = 8 then it will make 4 dataloaders of size 8 each 
    """
    max_len = 70 # just max number of tokens from LSTM    keep this line in here somewhere

    dataset = BertDataset(df , max_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, pin_memory=pin_memory, 
            num_workers=num_workers, drop_last=False, shuffle=True)
    return dataloader

Set our hyperparameters, and add in our metrics and criterion

In [ ]:
max_len = 70 # just max number of tokens from LSTM    keep this line in here somewhere
epoch = param_dict['epoch']
lr = param_dict['lr']
patience = param_dict['patience']
clip = param_dict['clip']
T_max = param_dict['T_max']
batch_size = param_dict['batch_size']
weight_decay = param_dict['weight_decay']
weights = param_dict['weights']
label2id = param_dict['label2id']
id2label = param_dict['id2label']

num_labels = model_param['output_dim']
input_dim = model_param['input_dim']
lstm_layers = model_param['lstm_layers']
hidden_layers = model_param['hidden_layers']

criterion = torch.nn.CrossEntropyLoss().to(rank)
Metric = Metrics(num_classes = num_labels, id2label = id2label , rank = rank)

Make sure to convert the dataframe to the dataloader and intialize the model

In [ ]:
df_train = prepare_dataloader(df_train, batch_size = batch_size )
df_val = prepare_dataloader(df_val, batch_size = batch_size )
df_test = prepare_dataloader(df_test , batch_size = batch_size)

model = BertClassifier(model_param).to(rank)

Train our model on the train and val dataloaders, along with our hyperparameters from above

In [ ]:
model = train_text_network(model, df_train, df_val, criterion , lr, epoch ,  weight_decay,T_max, Metric , patience , clip )

Our training loop is quite simple, all it does it just take in the batched inputs, run it through the model, then backpropogate, take a step with our optimizer and zero all the gradients.

Here we take in whatever our input and labels are and just run them through our model and get back our output

If we are in train or val we have the loss function work, else we dont use it

Then we get our predicts and labels and update them to our metrics so we can calculate F1, multi F1 Confusion matrices and many more


In [1]:
def get_statistics(input,label,model,criterion,total_loss, Metric):
    batch_loss = None
    label = label.to(f"cuda")

    if model.__class__.__name__ == "BertClassifier":
        mask = input['attention_mask'].to(f"cuda")
        input_id = input['input_ids'].squeeze(1).to(f"cuda")
        # print(f"input_id = {input_id}")
        output = model(input_id, mask)
    else:
        # pdb.set_trace()
        output = model(input) # shape was 16,128,2 , now its 16,2
    
    if criterion is not None:
        batch_loss = criterion(output, label.long())
        total_loss += batch_loss.item()
        
    # print("before update metrics" , flush = True)
    # print(f"output of BERT is {output} \n shape of output is {output.shape}\n")
    Metric.update_metrics(torch.argmax(output , dim = 1) , label.long())
    return batch_loss , total_loss

In one epoch is where we actually do our training. so we just get our statistics from above and just run our three functions

model.zero_grad()

train_batch_loss.backward()  # Back propagate

optimizer.step()

to learn

In [ ]:

def one_epoch(train_dataloader , model , criterion , optimizer, clip , Metric):
    total_loss_train = 0

    for train_input, train_label in tqdm(train_dataloader , desc="training"):
        # print("before getting Stats" , flush = True)
        train_batch_loss , total_loss_train  = get_statistics(train_input , train_label , model , criterion , total_loss_train , Metric)
        if clip is not None: # so gradients can beccome super big and super small, this clip is like a little flag that says we are too big or too small, 
            # lets cut it off here eg. clip is 50, if our gradients are 200 itll jsut make it 50 
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        model.zero_grad()
        train_batch_loss.backward()  # Back propagate
        optimizer.step()  # Run optimization step
    
    return  model , optimizer , train_batch_loss,total_loss_train/len(train_dataloader.dataset)


Here we just have our validation method to run both the validate and testing code as they are quite similar

In [ ]:

def validate(val_dataloader , model , criterion, Metric , name = "val" , location = "val"):
    total_loss_val = 0
    with torch.no_grad():
        for val_input, val_label in tqdm(val_dataloader, desc = "validate" if name == "val" else "testing" ):
            val_batch_loss , total_loss_val = get_statistics(val_input , val_label , model , criterion , total_loss_val  , Metric , name , location)

    return  val_batch_loss,total_loss_val/len(val_dataloader.dataset) 

Here is the general main frame to train and validate our code

We just go through each epoch and call the functions above

then from there we send our results to WANDB so we can look at them further for Hyper Parameter tuning

In [ ]:
def train_text_network(model, train_dataloader, val_dataloader, criterion , learning_rate, epochs , weight_decay,T_max, Metric , patience = None, clip = None  ) :  # TODO Fill in the information necessary as variables here.
    
    optimizer = AdamW(model.parameters(), lr = learning_rate , weight_decay=weight_decay)
    earlystop = EarlyStopping("",model,patience , model_name=model.__class__.__name__)
    scheduler = CosineAnnealingLR(optimizer , T_max=T_max)

    for epoch_num in tqdm(range(epochs), desc="epochs"):
        # model.train() # we want the model flag for training to be set true, this way the model knows its about to change
        optimizer.zero_grad()  # Zero out gradients before each epoch.
        
        # this is where we start to train our model
        model , optimizer , train_batch_loss,train_loss = one_epoch(train_dataloader ,  model , criterion , optimizer , clip , Metric ) 
        multiAcc , multiF1, multiRec, multiPrec , Acc, F1, Rec, Prec , _ = Metric.compute_scores("train")
        d1 = {
                "epoch": epoch_num,
                "train/batch_loss": train_batch_loss,
                "train/train_loss": train_loss,
                "train/acc": Acc,
                "train/precision": Prec,
                "train/recall" : Rec,
                "train/f1-score": F1,
            }
        print(f"\n in train \n Confusion Matrix = {_} \n")
        wandb.log({**d1 , **multiF1, **multiRec, **multiPrec, **multiAcc}) 
        Metric.reset_metrics()
        scheduler.step() 

        val_batch_loss,val_loss = validate(val_dataloader  , model , criterion , Metric)

        multiAcc , multiF1, multiRec, multiPrec , Acc, F1, Rec, Prec , _ = Metric.compute_scores("val")

        d1 = {
                "val/batch_loss": val_batch_loss,
                "val/total_loss_val": val_loss,
                "val/total_acc_val": Acc,
                "val/precision": Prec,
                "val/recall": Rec,
                "val/f1-score": F1,
                }
        print(f"\n in val \n Confusion Matrix = {_} \n")
        wandb.log({**d1 , **multiF1, **multiRec, **multiPrec, **multiAcc})
        Metric.reset_metrics()

        if patience is not None: # this is to make sure that gradietn descent isnt stuck in a local minima 
            if earlystop(model, val_loss):
                model = earlystop.best_state
                
    return model

Finally we evaluate our test set, which is quite similar to what we did for Validation

In [ ]:
evaluate_text(model, df_test, Metric)

In [ ]:
def evaluate_text(model, test_dataloader , Metric , location = "/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/Inference/bertTest.txt"):
    # model.eval() # this is where we put the flag for evaluating on so we dont change anything by accident
    name = "test"
    validate(test_dataloader  , model , None , Metric , name , location)
    multiAcc , multiF1, multiRec, multiPrec , Acc, F1, Rec, Prec , ConfusionMatrix = Metric.compute_scores("test")
    d1 = {
            "test/total_acc_test": Acc,
            "test/precision": Prec,
            "test/recall": Rec,
            "test/f1-score": F1,
            "test/ConfusionMatrix": ConfusionMatrix,
            }
    wandb.log({**d1 , **multiF1, **multiRec, **multiPrec, **multiAcc})    
    print(f"\n in TEST \n Confusion Matrix = {ConfusionMatrix} \n")    
  